# Notebook visualising the preprocessing of the 'moving_houses' dataset.

In [1]:
import pandas as pd
import numpy as np
import src.utils.preprocess_helper as pph

#### Reading the dataset.

In [10]:
move_houses = pd.read_csv('../../../data/move_houses_preprocessed.csv')

#### Preprocessing the `year` column and filtering the rows which does not belong to any Neighbourhood.

In [11]:
move_houses['year'] = move_houses['time_period'].astype(str).str[:4].astype(int)

move_houses = move_houses[['Neighbourhood', 'year', 'frequency', 'moving_type']].loc[
    ~pd.isna(move_houses['Neighbourhood'])]
move_houses.head()

,Neighbourhood,year,frequency,moving_type
0,Buitengebied Prinsenbeek,2013,2,Vertrek
1,Buitengebied Prinsenbeek,2013,1,Verhuizing
2,Buitengebied Prinsenbeek,2013,1,Verhuizing
3,Buitengebied Prinsenbeek,2013,1,Vertrek
4,Buitengebied Prinsenbeek,2013,2,Verhuizing


#### Predicting the future `frequency` values using the ARIMA Time Series model.
#### Additionally, the `moving_type` column was predicted again using the ARIMA Time Series model by applying an encoding.

In [151]:
future_years = [2020]
types = move_houses['moving_type'].unique()

future_data = pph.predict_future_yearly_data(move_houses.reset_index(), future_years, 'Neighbourhood', 'year',
                                             'frequency')

move_houses['moving_type'] = move_houses['moving_type'].apply(lambda x: np.squeeze(np.where(types == x)))
future_moving_type_data = pph.predict_future_yearly_data(
    move_houses.reset_index(), future_years, 'Neighbourhood', 'year',
    'moving_type')
future_data['moving_type'] = future_moving_type_data['moving_type'].astype(int).apply(lambda x: types[x])
move_houses['moving_type'] = move_houses['moving_type'].astype(int).apply(lambda x: types[x])

processed_df = pd.concat([move_houses, future_data]).groupby(['year', 'Neighbourhood', 'moving_type'])[
    'frequency'].mean().reset_index()

C:\Users\bobka\anaconda3\envs\buas\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\bobka\anaconda3\envs\buas\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\bobka\anaconda3\envs\buas\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\bobka\anaconda3\envs\buas\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive pa

#### Saving the preprocessed dataframe to a .csv file.

In [ ]:
pd.pivot_table(processed_df.loc[processed_df['year'] >= 2014], index=['Neighbourhood', 'year'],
               columns='moving_type',
               values='frequency',
               fill_value=0).reset_index().to_csv(
    '../../../data/preprocessed/move_houses_preprocessed.csv', index=False)